DEPENDENCIES

In [23]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import pandas as pd
import sqlite3
from sqlalchemy import create_engine , inspect
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, DateTime
import datetime as dt

SOUP EXTRACT

In [2]:
indeed  = 'https://www.indeed.com/jobs?q=data+analyst&l=#'
response = requests.get(indeed).text

In [3]:
soup = bs(response, 'html.parser')

In [4]:
jobCards = soup.find_all('div', class_= 'jobsearch-SerpJobCard')

TRANSFORM

In [6]:
#Indeed source
# Loop through returned Indeed soup object and create a list of dicts with values wanted for fields

jobList =[]

for job in jobCards:
    # Error handling
    try:
        # Identify and return title of listing
        title = job.find('a', class_='jobtitle').text.strip()
        company = job.find('span', class_='company').text.strip()
        location = job.find('div', class_ = 'location accessible-contrast-color-location').text.strip()
      

        # Print results only if title, price, and link are available
        if (title and company and location):

           # Dictionary to be inserted into list
            post = {
                'job_title': title,
                'company': company,
                'location': location
            }
            jobList.append(post)
            
    except AttributeError as e:
        pass

In [24]:
#Indeed List of Dicts create a df
df1 = pd.DataFrame(jobList)  

In [25]:
df1.head(5)

,job_title,company,location
0,CCoE CM Cyber Sit. Understanding Data Analyst,FITT Scientific LLC,"Fort Gordon, GA 30905"
1,SQL Data Analyst,Utility Trailer Manufacturing Company,"Atkins, VA 24311"
2,Pricing Analyst (Business Analyst),Bobcat Company,"Bismarck, ND"
3,Data Business Analyst,Payzer,"Charlotte, NC 28226 (Johnston Rd.-McAlpine area)"
4,Data Analyst,Private,"Aurora, CO 80011 (Jewell Heights - Hoffman Hei..."


In [ ]:
# Import CSV output  from splinter code to a dataframe
df2 = pd.read_csv('file') #Remco to populate

In [ ]:
# Concat df1 and df2 to df3

df3 = pd.DataFrame(df3df1.append(df2))

LOAD

In [26]:
# Create the posting class (the posting table that will be in the db)
class Posting(Base):
    __tablename__ = 'posting'
    id = Column(Integer, primary_key=True)
    job_title = Column(String(255))
    company = Column(String(255))
    location = Column(String(255))
    timestamp = Column(DateTime())


In [35]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///job_postings.db')
conn = engine.connect()
Base.metadata.create_all(engine)  #this should create the table based on the classes defined
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [37]:
# Iterate over df and add rows as records to table

for i in df3.itertuples():

    try:
        record = Posting(job_title = i.job_title, company = i.company, location = i.location,timestamp = pd.datetime.now().replace(microsecond=0) )
        session.add(record)
        session.commit()
 
    except:
        pass
    


TEST


In [38]:
#Insepect the database
inspector = inspect(engine)
inspector.get_table_names()

['posting']

In [39]:
columns = inspector.get_columns('posting')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
job_title VARCHAR(255)
company VARCHAR(255)
location VARCHAR(255)
timestamp DATETIME


In [40]:
postings = session.query(Posting)
for job in postings:
  print(job.job_title)

CCoE CM Cyber Sit. Understanding Data Analyst
SQL Data Analyst
Pricing Analyst (Business Analyst)
Data Business Analyst
Data Analyst
Data Analyst
Data Analyst
Data Analyst - 9867780
Data Analyst
CCoE CM Cyber Sit. Understanding Data Analyst
SQL Data Analyst
Pricing Analyst (Business Analyst)
Data Business Analyst
Data Analyst
Data Analyst
Data Analyst
Data Analyst - 9867780
Data Analyst


RESET DataBase

In [41]:
# Use this to clear out the db when necessary

#Base.metadata.drop_all(engine)

Troubleshooting Resources:

In [ ]:
# If needed to explore the soup object

#jobCard = soup.find('div', class_= 'jobsearch-SerpJobCard')
#print(jobCard.prettify())